In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from distfit import distfit
import re
from sympy import symbols, simplify, sin, cos, log, Abs, exp, expand_power_base, Mul, Rational, sqrt
from sympy.parsing.sympy_parser import parse_expr
import seaborn as sns
import copy
import json
from ydata_profiling import ProfileReport
import sys
import os
import bnlearn as bn
os.environ["JULIA_NUM_THREADS"] = "8"

# Add the directory containing 'thesis_utils.py' to the system path
sys.path.append(os.path.abspath('..'))  # If 'thesis_utils.py' is one level up
# Or use this if it's in the same directory as the notebook:
# sys.path.append(os.getcwd())

from thesis_utils import * 

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [38]:
import importlib
import thesis_utils  # or whatever your module is called

importlib.reload(thesis_utils)

from thesis_utils import *

In [2]:
data = pd.read_csv('C:/Users/svenl/vs_code_projects/hyperRealDataDescriber/data/credit_score/cleaned_credit_score_v2.csv')
data = data.drop(['ID', 'Customer_ID', 'Name', 'SSN', 'Type_of_Loan', 'Payment_Behaviour'], axis=1)

In [3]:
# Generate the profiling report
profile = ProfileReport(data, title="Credit Score Cleaned", explorative=True)

# Get the description object
desc = profile.get_description()

# Access the correlation matrix
correlation_matrix = desc.correlations["auto"]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
categorical_cols = get_categorical_columns(data)
distributions = fit_distributions(data)
marginals = convert_distfit_to_marginals(distributions, data)

In [17]:
data = deanonymize_column_names(data, rename)
data_syn = deanonymize_column_names(data_syn, rename)

In [18]:
data_syn = generate_synthetic_dataset(original_data=data, correlation_matrix=correlation_matrix, categorical_columns=categorical_cols, marginals=marginals, n_rows=37000, correlation_threshold=0.1, edge_strategy="random", sampling_strategy="local-chunks")

In [19]:
data_syn = data_syn[data.columns.to_list()]

In [6]:
data_syn, rename = anonymize_column_names(data_syn)

In [20]:
# Generate the profiling report
profile_syn = ProfileReport(data_syn, title="Credit Score Cleaned SYN", explorative=True)

# Get the description object
desc_syn = profile_syn.get_description()

# Access the correlation matrix
correlation_matrix_syn = desc_syn.correlations["auto"]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

In [21]:
plot_correlation_matrices(correlation_matrix, correlation_matrix_syn, "Real", "Synthetic")

In [22]:
data, rename = anonymize_column_names(data)
data_syn, rename = anonymize_column_names(data_syn)

from sdmetrics.reports.single_table import QualityReport
from sdmetrics.single_table import NewRowSynthesis

metadata = {
    "columns": {
        "X1": {"sdtype": "categorical"},
        "X2": {"sdtype": "numerical"},
        "X3": {"sdtype": "categorical"},
        "X4": {"sdtype": "numerical"},
        "X5": {"sdtype": "numerical"},
        "X6": {"sdtype": "numerical"},
        "X7": {"sdtype": "numerical"},
        "X8": {"sdtype": "numerical"},
        "X9": {"sdtype": "numerical"},
        "X10": {"sdtype": "numerical"},
        "X11": {"sdtype": "numerical"},
        "X12": {"sdtype": "numerical"},
        "X13": {"sdtype": "numerical"},
        "X14": {"sdtype": "categorical"},
        "X15": {"sdtype": "numerical"},
        "X16": {"sdtype": "numerical"},
        "X17": {"sdtype": "numerical"},
        "X18": {"sdtype": "categorical"},
        "X19": {"sdtype": "numerical"},
        "X20": {"sdtype": "numerical"},
        "X21": {"sdtype": "numerical"},
    }
}

report = QualityReport()
report.generate(data, data_syn, metadata)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 21/21 [00:00<00:00, 47.55it/s]|
Column Shapes Score: 93.65%

(2/2) Evaluating Column Pair Trends: |██████████| 210/210 [00:01<00:00, 128.59it/s]|
Column Pair Trends Score: 91.84%

Overall Score (Average): 92.75%



In [20]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Score,Error
0,X1,TVComplement,1.273357e-09,None
1,X2,KSComplement,1.734595e-01,None
2,X3,TVComplement,1.095679e-06,None
3,X4,KSComplement,0.000000e+00,None
4,X5,KSComplement,0.000000e+00,None
5,X6,KSComplement,0.000000e+00,None
6,X7,KSComplement,2.612719e-01,None
7,X8,KSComplement,9.556066e-01,None
8,X9,KSComplement,2.702703e-05,None
9,X10,KSComplement,0.000000e+00,None


In [21]:
report.get_details(property_name='Column Pair Trends')

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation,Error
0,X1,X2,ContingencySimilarity,0.000000e+00,NaN,NaN,None
1,X1,X3,ContingencySimilarity,-4.440892e-16,NaN,NaN,None
2,X1,X4,ContingencySimilarity,0.000000e+00,NaN,NaN,None
3,X1,X5,ContingencySimilarity,0.000000e+00,NaN,NaN,None
4,X1,X6,ContingencySimilarity,0.000000e+00,NaN,NaN,None
...,...,...,...,...,...,...,...
205,X18,X20,ContingencySimilarity,NaN,NaN,NaN,TypeError: ufunc 'isfinite' not supported for ...
206,X18,X21,ContingencySimilarity,NaN,NaN,NaN,TypeError: ufunc 'isfinite' not supported for ...
207,X19,X20,CorrelationSimilarity,NaN,NaN,NaN,TypeError: unsupported operand type(s) for +: ...
208,X19,X21,CorrelationSimilarity,NaN,NaN,NaN,TypeError: unsupported operand type(s) for +: ...
